Analysis of financial statement

References
- YahooFinancials tutorial : https://www.analyticsvidhya.com/blog/2021/06/download-financial-dataset-using-yahoo-finance-in-python-a-complete-guide/
- doc : https://pypi.org/project/yahoofinancials/

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import sys
import time
import yahoo_fin.stock_info as si

from datetime import timedelta, datetime
from sklearn.preprocessing import MinMaxScaler
from yahoofinancials import YahooFinancials as yf

In [ ]:
print(f"python version : {sys.version}")
print(f"pandas version : {pd.__version__}")
print(f"numpy version : {np.__version__}")

In [ ]:
air_canada = yf("AC.TO")
data = air_canada.get_historical_price_data(start_date='2021-10-01', end_date='2023-04-13',
                                     time_interval='daily')

In [ ]:
airlines = {"air_canada":"AC.TO",
            "delta":"DAL",
            'korean_air':"003490.KS","Asiana":"020560.KS"}
airline_tickers = airlines.values()

In [ ]:
end_date = datetime.today().strftime('%Y-%m-%d')
st_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
hist_data = yf(airline_tickers).get_historical_price_data(start_date=st_date, end_date=end_date,
                                              time_interval='daily')

In [ ]:
stock_price_df_dict = {}
for airline_nm, ticker in airlines.items():
    print(airline_nm, ticker)
    df = pd.DataFrame(hist_data[ticker]['prices'])
    df = (df.drop(columns=['date'])
        .rename(columns={"formatted_date":"date"})
        .set_index('date')
        .sort_index(ascending=True))
    df.index = pd.to_datetime(df.index)
    df['currency'] = hist_data[ticker]['currency']
    stock_price_df_dict[ticker] = df

In [ ]:
"""
Data transformation

- Since all stocks are from different stock exchanges, and have different range, apply scaling.


What to see:
1. See if stocks from similar stock exchange tend to move in similar direction
2. Since all stocks are stocks of airline company, do they move simlarily despite diff stock exchange? 
"""
def apply_minmax(df):
    mm_scaler = MinMaxScaler()
    num_cols = df.select_dtypes(include=['float', 'int']).columns
    df[num_cols] = mm_scaler.fit_transform(df[num_cols])
    return df

stock_price_df_dict_trsfmd = {ticker:apply_minmax(df.copy()) for ticker, df in stock_price_df_dict.items()}

In [ ]:
fig = go.Figure()
df = stock_price_df_dict_trsfmd['AC.TO']
fig.add_trace(
    go.Scatter(
        x= df.index,
        y= df['high'],
        name='AC.TO'))
    
fig.update_layout(
    title='Airline stocks comparison [high]')
fig.show()

In [ ]:
fig = go.Figure()
df = stock_price_df_dict['AC.TO']
fig.add_trace(
    go.Scatter(
        x= df.index,
        y= df['high'],
        name='AC.TO'))
    
fig.update_layout(
    title='Airline stocks comparison [high]')
fig.show()

In [ ]:
fig = go.Figure()
for ticker, df in stock_price_df_dict_trsfmd.items():
    fig.add_trace(
        go.Scatter(
            x= df.index,
            y= df['high'],
            name=ticker))
    
fig.update_layout(
    title='Airline stocks comparison [high]')
fig.show()

## EDA

### Stock movement comparison

In [ ]:
"""
See if stocks of similar domain tend to move together.
  If not, see what the differences are, for example even if the company is motor company it might have lot 
  of tangible assets that might be driving the stock price up.
"""

In [ ]:
fig = go.Figure(data=[
    go.Candlestick(x=df.index,
    open=df['open'],
    high=df['high'],
    low=df['low'],
    close=df['close'])])

fig.update_layout(
    title='AC.TO',
    yaxis_title='USD'
)
fig.show()

In [ ]:
air_canada.get_financial_data()

In [ ]:
balance_sheets = air_canada.get_financial_stmts(frequency='quarterly', statement_type='balance')
income_stmts = air_canada.get_financial_stmts(frequency='quarterly', statement_type='income')

In [ ]:
dfs = []
for inc_stmt in income_stmts['incomeStatementHistoryQuarterly']['AC.TO']:
    dfs.append(pd.DataFrame(inc_stmt).T)
income_stmt_df = pd.concat(dfs)

dfs = []
for q_stats in balance_sheets['balanceSheetHistoryQuarterly']['AC.TO']:
    dfs.append(pd.DataFrame(q_stats).T)
bal_sheet_df = pd.concat(dfs)

In [ ]:
financial_stmt_df = pd.concat([income_stmt_df, bal_sheet_df], axis=1).sort_index(ascending=True)
financial_stmt_df.index = pd.to_datetime(financial_stmt_df.index)

In [ ]:
# ? find out way to left join but then if right table do not match, join with previous day.

# historical dataset does not have date that match fin_stmt therefore shifted +1 one day to join.
financial_stmt_df = (financial_stmt_df
                      .merge(df
#                              .shift(periods=1, freq='D')
                             ,how='left', left_index=True, right_index=True)
                      .sort_index(ascending=True))

In [ ]:
financial_stmt_df['p/e ratio'] = (financial_stmt_df['high'] / 
                                  (financial_stmt_df['netIncome'] / financial_stmt_df['shareIssued'])
                                 )

In [ ]:
financial_stmt_df['netIncome'] / financial_stmt_df['shareIssued']

In [ ]:
financial_stmt_df['p/e ratio']

In [ ]:
total_revenue=16_556_000
cost_of_revenue = 13_757_000 #same as COGS : costs associated with producing companys products
# in air canda this includes : FUEL, maintenance&repairs, other cost of revenue(???), etc...
opearting_expenses = 2_982_000 # indirect costs such as wages, rents, insurance, marketing, etc...
gross_profit =  2_799_000 # revenue - COGS

In [ ]:
book_value = assets - liabilities
PBR = marker_price_per_share / book_value_per_share

earnings_per_share = (netincome - preferred_dividends) / Average oustanding common shares
PER = marker_price_per_share / earnings_per_share

## Air canada analysis

봐야 할것:

0. PER, PBR, 이익곡선 vs 주가 곡선 : 같은 분야의 기업들과 비교
1. 재무 상태, 현금 보유량
2. 기관투자자, 내부자 비중
3. 재고
4. 수요,공급 관계.
5. 신규로 진입하는 기업 a.k.a. 경쟁자 분석.

In [ ]:
airlines = {"air_canada":"AC.TO",
            "delta":"","westjet":"","lufthansa":"",
            "qatar":"","singapore_air":"",
            'korean_air':"","Asiana":"",
            "emirates":"",}

In [ ]:
balance_sheet = balance_sheet.T.sort_values("endDate")

In [ ]:
balance_sheet.T

# Timeseries forecasting